### Imports, Environment Setup

In [10]:
from sympy import *
from mpmath import degrees, radians
from sympy.matrices import Matrix, eye
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from pprint import pprint

In [2]:
init_printing(use_latex='mathjax')

In [8]:
# Conversion Factors - Note that this is using the symbolic definition of pi from SymPy 
rtd = 180./pi # radians to degrees
dtr = pi/180. # degrees to radians

In [5]:
# Assembles a translation/position column vector in 3d, option for output in homogeneous coordinates 
def col(input_tuple, h=0):
    matrixArg = [[item] for item in input_tuple]
    if h: matrixArg.append([1])
    return Matrix(matrixArg)

# # TEST:
# t = col((1, 0, 0))
# th = col((1, 0, 0), h=1)
# show(t)
# show(th)

In [6]:
### Create symbols for DH parameters
q1, q2, q3, q4, q5, q6, q7 = symbols('q1:8') # theta angles
d1, d2, d3, d4, d5, d6, d7 = symbols('d1:8')
a0, a1, a2, a3, a4, a5, a6 = symbols('a0:7')
alpha0, alpha1, alpha2, alpha3, alpha4, alpha5, alpha6 = symbols('alpha0:7')

### Create symbols for end effector target coordinates in base_link reference frame

x_target, y_target, z_target = symbols('x_target y_target z_target')

### Create symbols for wrist center location in base frame 

wrist_x, wrist_y, wrist_z = symbols('wrist_x wrist_y wrist_z')

# s_wrist = cylindrical radius from Z1 to the wrist center, w.
# s_wrist = sqrt(wrist_x**2 + wrist_y**2)

In [7]:
# Define the symbol dictionary which contains lookup values for the system
# ATTENTION! Note that joints 4-6 share a common origin, actually located at the wrist center, 
# at the intersection of their Z axes.
dh_parameter = {alpha0:     0,  a0:      0,  d1:  0.75,  
                alpha1: -pi/2,  a1:   0.35,  d2:     0,  q2: q2 - pi/2,
                alpha2:     0,  a2:   1.25,  d3:     0,  
                alpha3: -pi/2,  a3: -0.054,  d4:  1.50, 
                alpha4:  pi/2,  a4:      0,  d5:     0, 
                alpha5: -pi/2,  a5:      0,  d6:     0, 
                alpha6:     0,  a6:      0,  d7: 0.303,  q7: 0
                }

# # s is the substitutions dictionary
# s = {alpha0:     0,  a0:      0,  d1:  0.75,  q1: atan2(wcP_y, wcP_x),
#      alpha1: -pi/2,  a1:   0.35,  d2:     0,  q2: q2 - pi/2,
#      alpha2:     0,  a2:   1.25,  d3:     0,  
#      alpha3: -pi/2,  a3: -0.054,  d4:  1.50, 
#      alpha4:  pi/2,  a4:      0,  d5:     0, 
#      alpha5: -pi/2,  a5:      0,  d6:     0, 
#      alpha6:     0,  a6:      0,  d7: 0.303,  q7: 0
#      }

# dh_parameter = {alpha0:     0,  a0:      0,  d1:  0.75,  
#                 alpha1: -pi/2,  a1:   0.35,  d2:     0,  q2: q2 - pi/2, # ?????
#                 alpha2:     0,  a2:   1.25,  d3:     0,  
#                 alpha3: -pi/2,  a3: -0.054,  d4:  1.50, 
#                 alpha4:  pi/2,  a4:      0,  d5:     0, 
#                 alpha5: -pi/2,  a5:      0,  d6:     0, 
#                 alpha6:     0,  a6:      0,  d7: 0.303,  q7: 0
#                 }

In [7]:
joint_decomp = {'urdf': {},
                'DH_strict': {},
                'DH_actual': {}
                }

joint_decomp['urdf']['x'] = [0, 0, 0.35, 0.35, 1.31, 1.85, 2.043, 2.153, 2.303]
joint_decomp['urdf']['y'] = [0, 0, 0, 0, 0, 0, 0, 0, 0]
joint_decomp['urdf']['z'] = [0, 0.33, 0.75, 2.0, 1.946, 1.946, 1.946, 1.946, 1.946]

joint_decomp['DH_strict']['x'] = [0, 0, 0.35, 0.35, 1.85, 1.85, 1.85, 2.153, 2.303]
joint_decomp['DH_strict']['y'] = [0, 0, 0, 0, 0, 0, 0, 0, 0]
joint_decomp['DH_strict']['z'] = [0, 0.75, 0.75, 2.0, 1.946, 1.946, 1.946, 1.946, 1.946]

joint_decomp['DH_actual']['x'] = [0, 0, 0.35, 0.35, 1.31, 1.85, 2.043, 2.153, 2.303]
joint_decomp['DH_actual']['y'] = [0, 0, 0, 0, 0, 0, 0, 0, 0]
joint_decomp['DH_actual']['z'] = [0, 0.75, 0.75, 2.0, 1.946, 1.946, 1.946, 1.946, 1.946]

relative_joints = {0: {'urdf': (0, 0, 0), # base_link origin
                       'dh':   (0, 0, 0)},
                   
                   1: {'urdf': (0, 0, 0.33),
                       'dh':   (0, 0, 0.75)},
                   
                   2: {'urdf': (0.35, 0, 0.42),
                       'dh':   (0.35, 0, 0)},
                   
                   3: {'urdf': (0, 0, 1.25),
                       'dh':   (0, 0, 1.25)},
                   
                   4: {'urdf': (0.96, 0, -0.054),
                       'dh':   (1.5, 0, -0.054)},
                   
                   5: {'urdf': (0.54, 0, 0),
                       'dh':   (0, 0, 0)},
                   
                   6: {'urdf': (0, 0, 0),
                       'dh':   (0, 0, 0)},
                   
                   7: {'urdf': (0.11, 0, 0),
                       'dh':   (0.11, 0, 0)}, # gripper_joint origin
                   
                   8: {'urdf': (0, 0, 0),
                       'dh':   (0, 0, 0)}  # extended finger target origin
}
                   
joints = {0: {'urdf': (0, 0, 0)}, # base_link origin
          1: {'urdf': (0, 0, 0.33)},
          2: {'urdf': (0.35, 0, 0.75)},
          3: {'urdf': (0.35, 0, 2.00)},
          4: {'urdf': (1.31, 0, 1.946)},
          5: {'urdf': (1.85, 0, 1.946)},
          6: {'urdf': (2.043, 0, 1.946)},
          7: {'urdf': (2.153, 0, 1.946)}, # gripper_joint origin
          8: {'urdf': (2.303, 0, 1.946)}} # extended finger target origin

zero_pose_joint_axes = {1: 'z',
                        2: 'y',
                        3: 'y',
                        4: 'x',
                        5: 'y',
                        6: 'x'
                        }

## I. The Inverse Positioning Problem

### A. Input Target

In [8]:
default_target_x = 2.30286
default_target_y = 0.0
default_target_z = 1.94658

t = {x_target: default_target_x,
     y_target: default_target_y, 
     z_target: default_target_z
     }

def retarget(x=None, y=None, z=None):
    global t
    if not (x or y or z):
        t[x_target] = default_target_x
        t[y_target] = default_target_y
        t[z_target] = default_target_z
        return
    else:
        if x: t[x_target] = x
        if y: t[y_target] = y
        if z: t[z_target] = z
        return

# # TEST:
# print(t)
# retarget(1,2,1.5)
# print(t)
# retarget()
# print(t)

retarget()

### B. Constrain Angle of Approach

For the purposes of deriving the general form IK solution, this should be defined symbolically.

The vector 'p_placeholder' is an x_0 unit vector, i.e. pointing parallel to the x-axis of the base link. 

This definition for p_placeholder was chosen to reflect the geometric constraints of the pick and place scene (specifically the accessibility of the shelves and the alignment of the target objects).

The assertion that the end effector must be at this specific orientation may be more rigid than necessary to ensure smooth operation, but *some* specific orientation must be chosen to constrain the problem, and aligning the end effector with a surface normal vector on the shelf's openings should go a long way towards maximizing the accessible volume of any shelf (if not perfectly optimizing access, which I believe it will, in fact, do - but I haven't tried to prove this).

Furthermore, with our end effector aligned along the depth dimension of the shelves, it is natural enough to specify a location on the opening's 'surface' at which to engage the automatic reaching and grasping routines.

In [10]:
shelf_interaction_orientation = col((1, 0, 0))
dropoff_orientation = col((0, 1, 0))

# end_eff_unitvector set to pp_placeholder as default for this project, as it is already ideal for testing. 
# Could be made symbolic to generalize the process, to simply set to a different default for other configurations.
end_effector_unitvector = shelf_interaction_orientation

# All three of these distances are measured along the Z6 axis; 
# the distances are displacements relative to the previous variable, 
# with joint_6 measured from the wrist center. 
joint_6_distance = 0.193
gripper_joint_distance = 0.11
finger_joint_distance = 0.15 # not included in example value

with_grippers = True

# End effector length spans distance from wc to target point
end_effector_length = joint_6_distance + gripper_joint_distance + (int(with_grippers) * finger_joint_distance)

# end_effector_vector points from wc to target point
end_effector_vector = end_effector_length * end_effector_unitvector

# Rotation of the Positioning Plane: Theta_1

Since joints 0-5 are confined to the same vertical plane, and these are sufficient to solve the inverse positioning problem, I will call said plane the 'positioning plane.' With the wrist center determined, this plane's orientation about the Z1 axis is fully constrained. This orientation is determined by the value of theta_1, which is calculated below:

In [66]:
# retarget(2.2, 1.7, 1.16)
retarget()

wrist_x = x_target - end_effector_vector[0]
wrist_y = y_target - end_effector_vector[1]
wrist_z = z_target - end_effector_vector[2]
wrist_x, wrist_y, wrist_z

(x_target - 0.453, y_target, z_target)

In [67]:
theta_1 = atan2(wrist_y, wrist_x)
theta_1

atan2(y_target, x_target - 0.453)

In [68]:
# Now that we have theta1, we will add it to our lookup dict
joint_angle_equations[q1] = theta_1
joint_angles[q1] = joint_angle(1)

joint_angle_equations[q1], joint_angles[q1]
# dh_parameter[q1], angle_equations[q1]

(atan2(y_target, x_target - 0.453), 0)

In [69]:
# wrist_s = sqrt(wrist_x**2 + wrist_y**2).evalf(subs=t)
wrist_s = sqrt(wrist_x**2 + wrist_y**2)
wrist_s, wrist_s.subs(t)

⎛   _________________________________         ⎞
⎜  ╱         2                     2          ⎟
⎝╲╱  y_target  + (x_target - 0.453)  , 1.84986⎠

# Solving Theta_2 & Theta_3

In [70]:
# Regardless of our choice of method, we will need the following values to be loaded:
# 
# radius_j2_j3 = dh_parameter[a2]

# The magnitude of the vector pointing from joint_3 to joint_5, although our DH parameter table
# files this location under the transformation to joint_4 coordinates, since joints 4 and up 
# share an origin at the wrist center, coincident with joint 5.
radius_j5_j3 = sqrt(dh_parameter[a3]**2 + dh_parameter[d4]**2)

# displacements in the positioning plane between the wrist center and joint_2.
delta_s = wrist_s - dh_parameter[a1] # dh_parameter[a1] = 0.35 (meters)
delta_z = wrist_z - dh_parameter[d1] # dh_parameter[d1] = 0.75 (meters)

radius_j5_j3, delta_s, delta_z

⎛                     _________________________________                       
⎜                    ╱         2                     2                        
⎝1.50097168527591, ╲╱  y_target  + (x_target - 0.453)   - 0.35, z_target - 0.7

 ⎞
 ⎟
5⎠

## The SSS Triangle Method:
There is a slight hiccup here, the vertical difference between joints 3 and 4. This  of 
Since the articulation of joint 4 is revolute about the axis between itself and joint 5 (a.k.a. the wrist center), and since both joint origins lie in plane with joint 3 AND joint 2, finding ain plane with the wrist center - **BUT ONLY APPROXIMATELY; SEE CORRECTION!** - the 
The origins of joints 2, 3, and 5 all lie in the
Let a, b, c represent sides of the triangle 

The differences between the coordinates of joint_2 and joint_5 are important parameters for solving the SSS triangle, ABC, described above. These differences are also the x and y coordinates of the wrist center in the joint 2 reference frame. The values are denoted here as delta_s and delta_z.

With the arm rotated into the positioning plane, the cylindrical radius to joint_2 is a constant 0.35m along s.
Similarly, joint_2 is always a constant 0.75m above the base link's z=0 plane, regardless of any other parameters.

Together, this means that joint_2 traverses a circle of radius 0.35m, fixed at 0.75m off the ground plane.
Setting theta_1 and thus the orientation of s effectively selects a point for j2 on this circle, and that is the origin for the coordinates of the SSS Triangle method.

[//]: # (Image References)

[image1]: ./Pick-and-Place/SSS_Triangle.png

![alt text][image1]

In [71]:
# Let A, B, C name the vertices of the triangle described by the origins of joints 2, 3, and 4.
# let the symbols A and B represent the interior angles of triangle ABC at their synonymous vertices.
A, B, C = symbols('A B C')
# C is not strictly needed, but it may be useful for appraising whether a given pose is possible.
# If C comes back imaginary, this would indicate an unviable solution.

# Note that vertex angle C is not given a symbol; it simply isn't necessary, as it wouldn't be used in any later step.
# A, B = symbols('A B')

# let a, b, c represent side lengths of ABC, named conventionally (opposite their angles)
a, b, c = symbols('a b c')

# Here, A is the vertex located at joint_2, B at joint_3, and C at joint_5 (shared origin with joints 4-6)

the x-axis of the joint_1 reference frame, called here 's.' The s-axis is determined by the wrist center: it is parallel to the projection of a vector from the base_link origin to the wrist center. Referring to this dimension or its values as 's' may be unnecessary complication, but it emphasizes the profound cylindrical symmetry of manipulator configurations.


Determining theta_1 doesn't simply move us closer to solving for all the joint angles; it permits us to rotate the coordinate system the appropriate amount to locate it into the sz-plane required by the wrist center.

In [72]:
# Initially we have all side lengths, so let's prepare a dict:
sides = {}
# ...but only one leg has a known location/orientation: side b, which is coincident with a vector pointing from j2 to j5.
# This vector ( we'll just call it "b_vector") ultimately determines the existence and nature of any solutions.
# The side length b is the magnitude of b_vector:
# b = sqrt(delta_s**2 + delta_z**2)

# sides[b] = sqrt(delta_s**2 + delta_z**2).evalf(subs=t)
sides[b] = sqrt(delta_s**2 + delta_z**2)

# a and c are fixed distances, a the distance between joints 3 and 5 (=1.25m) and b between 2 and 3.
# These are both defined above, as radius_j5_j3 and radius_j2_j3 respectively
sides[a] = radius_j5_j3
sides[c] = dh_parameter[a2]

sides

⎧                             ________________________________________________
⎪                            ╱                                                
⎨                           ╱                       ⎛   ______________________
⎪                          ╱                    2   ⎜  ╱         2            
⎩a: 1.50097168527591, b: ╲╱    (z_target - 0.75)  + ⎝╲╱  y_target  + (x_target

_____________________         ⎫
                   2          ⎪
___________       ⎞           ⎬
         2        ⎟           ⎪
 - 0.453)   - 0.35⎠  , c: 1.25⎭

In [73]:
# Solving SSS Triangles is fun and simple!
A = acos((b**2 + c**2 - a**2)/(2 * b * c)).subs(sides)
B = acos((a**2 + c**2 - b**2)/(2 * a * c)).subs(sides)
C = acos((a**2 + b**2 - c**2)/(2 * a * b)).subs(sides)
# A, B

In [74]:
A = A.evalf(subs=t)
B = B.evalf(subs=t)
C = C.evalf(subs=t)

# Calculate numerical error between expected sum of interior angles and the sum of our results.
num_error = N(pi - (A + B + C))

# If A, B, or C are imaginary or complex, there is no physical solution.
# If num_error is not approximately zero, then there is likely an error upstream. 
A, B, C, num_error

(0.897530287632943, 1.53506992311703, 0.708992442839817, -3.21624529935327e-16
)

In [75]:
# The rotation of b_vector about Z2 (common normal of s and Z1) sets the orientation of the triangle ABC, 
# which in turn determines various angular terms in the formulas for theta_2 and theta_3.
# One such term is wc_pitch, the angle between b_vector and the s-axis:
wc_pitch = atan2(delta_z, delta_s)
wc_pitch, wc_pitch.evalf(subs=t)

⎛     ⎛                    _________________________________       ⎞          
⎜     ⎜                   ╱         2                     2        ⎟          
⎝atan2⎝z_target - 0.75, ╲╱  y_target  + (x_target - 0.453)   - 0.35⎠, 0.673394

         ⎞
         ⎟
653635177⎠

In [76]:
theta_2 = pi/2 - wc_pitch - A

In [77]:
j4_correction = atan2(0.054, 1.5)
j4_correction

0.0359844600820516

In [78]:
theta_3 = pi/2 - B - j4_correction

In [79]:
theta_2, theta_3

⎛       ⎛                    _________________________________       ⎞        
⎜       ⎜                   ╱         2                     2        ⎟        
⎜- atan2⎝z_target - 0.75, ╲╱  y_target  + (x_target - 0.453)   - 0.35⎠ - 0.897
⎝                                                                             

                                       ⎞
               π                      π⎟
530287632943 + ─, -1.57105438319909 + ─⎟
               2                      2⎠

In [80]:
# dh_parameter[q2] = theta_2.evalf(subs=t)
# dh_parameter[q2] = theta_2.evalf(subs=t)
# dh_parameter[q2]

joint_angle_equations[q2] = theta_2
joint_angles[q2] = joint_angle(2)

joint_angle_equations[q2], joint_angles[q2]

⎛       ⎛                    _________________________________       ⎞        
⎜       ⎜                   ╱         2                     2        ⎟        
⎜- atan2⎝z_target - 0.75, ╲╱  y_target  + (x_target - 0.453)   - 0.35⎠ - 0.897
⎝                                                                             

                                       ⎞
               π                       ⎟
530287632943 + ─, -0.000128614473223027⎟
               2                       ⎠

In [81]:
# dh_parameter[q3] = theta_3.evalf()
# dh_parameter[q3]

joint_angle_equations[q3] = theta_3
joint_angles[q3] = joint_angle(3)

joint_angle_equations[q3], joint_angles[q3]

⎛                    π                       ⎞
⎜-1.57105438319909 + ─, -0.000258056404188461⎟
⎝                    2                       ⎠

In [82]:
# This cell is useful for comparing with the forward_kinematics.rviz sim; the TF position of link_5 should be coincident
# with the location of 'wrist' once the angles of joints 1-3 are set to q1, q2, and q3 respectively,
# referred to here as the 'position_angles,' read top to bottom.
wrist = col((wrist_x, wrist_y, wrist_z)).evalf(subs=t)
positioning_joint_angles = col((('theta_1', joint_angles[q1]),
                                ('theta_2', joint_angles[q2]),
                                ('theta_3', joint_angles[q3])
                               ))


wrist, positioning_joint_angles

⎛⎡1.84986⎤  ⎡          (θ₁, 0)          ⎤⎞
⎜⎢       ⎥  ⎢                           ⎥⎟
⎜⎢   0   ⎥, ⎢(θ₂, -0.000128614473223027)⎥⎟
⎜⎢       ⎥  ⎢                           ⎥⎟
⎝⎣1.94658⎦  ⎣(θ₃, -0.000258056404188461)⎦⎠

In [83]:
# Since angles A, B, C are positive definite, and wc_pitch is measured CCW from the horizontal (opposite sign to theta_2),
# the angle (B + wc_pitch) is effectively wc_pitch, with some as-yet-undetermined positive phase shift.
# 
# Theta_2 is initially offset by a phase of 90 degrees in the positive gamma direction

### Simultaneous Circle Solver Method? Describe?

In [84]:
j2_0 = col((dh_parameter[a1], dh_parameter[d1]))

s_3, z_3 = symbols('s_3, z_3')
j3_0 = col((s_3, z_3)) + j2_0
# j5 = col(wrist_s, 0, wrist_z)

j5_0 = col((wrist_s, wrist_z)).subs(t)

In [85]:
print("In O0 coordinates: ")
j2_0, j3_0, j5_0

In O0 coordinates: 


⎛⎡0.35⎤  ⎡s₃ + 0.35⎤  ⎡1.84986⎤⎞
⎜⎢    ⎥, ⎢         ⎥, ⎢       ⎥⎟
⎝⎣0.75⎦  ⎣z₃ + 0.75⎦  ⎣1.94658⎦⎠

In [86]:
j2 = col((0, 0))       
j3 = col((s_3, z_3))
j5 = j5_0 - j2_0

In [87]:
print("In O2 coordinates: ")
j2, j3, j5

In O2 coordinates: 


⎛⎡0⎤  ⎡s₃⎤  ⎡1.49986⎤⎞
⎜⎢ ⎥, ⎢  ⎥, ⎢       ⎥⎟
⎝⎣0⎦  ⎣z₃⎦  ⎣1.19658⎦⎠

In [88]:
s3_expr_1 = dh_parameter[a2] * sin(q2)
z3_expr_1 = dh_parameter[a2] * cos(q2)

s3_expr_1, z3_expr_1

(1.25⋅sin(q₂), 1.25⋅cos(q₂))

In [89]:
# gamma_5 is an intermediate angle in this calculation.
# For now, it represents the deflection of the joint 5 radiant from the horizontal,
# as shown in illustration. 
# Behaves identically to theta_2 for ease of explanation. 
# This is hard to visualize, add illustration!

In [90]:
gamma_5 = Symbol('gamma_5')

s3_expr_2 = j5[0] + (radius_j5_j3 * -cos(gamma_5))
z3_expr_2 = j5[1] + (radius_j5_j3 * sin(gamma_5))

s3_expr_2, z3_expr_2

(-1.50097168527591⋅cos(γ₅) + 1.49986, 1.50097168527591⋅sin(γ₅) + 1.19658)

In [91]:
# Since expressions 1 and 2 for both s_3 and z_3 must be equal for a solution to exist, subtracting the second expression
# from the first should make each of these combined expressions equal to zero. 
# This format is now suitable as input to SymPy's equation solver.
s3_zero_form = s3_expr_1 - s3_expr_2
z3_zero_form = z3_expr_1 - z3_expr_2

s3_zero_form, z3_zero_form

(1.25⋅sin(q₂) + 1.50097168527591⋅cos(γ₅) - 1.49986, -1.50097168527591⋅sin(γ₅) 
+ 1.25⋅cos(q₂) - 1.19658)

In [92]:
# solving these expressions simultaneously gives a pair of numerical values for q2 and gamma_5 - 
# if solution a solution exists.
# But even then, there's a catch: generally, there are two possible solutions in the solution set.
soln_set = solve([s3_zero_form, z3_zero_form], (q2, gamma_5))

soln_set

[(-0.000128614473224601, 0.0355977892046402), (1.79493196079266, -1.3823870964
7499)]

Now we must calculate theta_3 given theta_2 and gamma_5. This is a subtle and confusing step, since we must be 

In [93]:
def theta3from(theta_2, gamma_5):
    # when gamma_5 = theta_2, theta_3 == 0
    # when gamma_5 = 0, theta_3 == -theta_2
    theta_3 = theta_2 - gamma_5
    return theta_3

theta3from(pi/4, pi/4), theta3from(0, pi/2), theta3from(pi/2, 0), theta3from(0, 0)

⎛   -π   π   ⎞
⎜0, ───, ─, 0⎟
⎝    2   2   ⎠

## Joint Range Enforcement

Before going forward, we should vet any and all solutions to ensure that they are viable commands for the robot. trying to choose between two solutions, or for that matter, accepting the miraculous appearance of a unique solution, First, we must resolve the value of theta_3. This must be calculated from gamma_5 and geometric methods.

In [94]:
# # joint 2 ranges from -45 degrees to +85 degrees;
# j2_min = -45 * dtr
# j2_max = 85 * dtr

# # joint 3 ranges from -210 degrees to +65 degrees;
# j3_min = -210 * dtr
# j3_max = 65 * dtr

# viable_soln_set = []

# for solution in soln_set:
#     if j2_max > solution[0] > j2_min:
#         if j3_max > 

### Discriminating Between Multiple Viable Solutions

From a control and integrity standpoint, it makes sense to keep the robot's center of mass as near to the Z1 axis as possible, which could be calculated more delicately using the inertial information in the URDF file. This would be a bit of an undertaking, to say the least.

Another general consideration is similarity to the current pose. This is much easier to assess.

In [95]:
# Interestingly, as long as we solve for 

## Joints 4-6: The Orientation Problem

The process of conforming the wrist links onto the now-constrained arm links breaks down into a few separate elements. 
    
    - Account for the slight misalignment between j3-j5 axis and rotation axis of joint_4
    - Find theta_4 such that joint_5's DoF can access the target point.
    - Find theta_6 such that the end effector is correctly oriented relative to the target.


We do, however, have some parts of this process partially complete!
The angles of joints 4 and 5 are ,

In [96]:
def vector_cross_angle(first_vector, second_vector):
    # The vector arguments should be Matrix objects with three elements
    unit_1 = first_vector.normalized()
    unit_2 = second_vector.normalized()
    cross_vector = unit_1.cross(unit_2)
    return N(asin(cross_vector.norm()))

def vector_cross_unit(first_vector, second_vector):
    # The vector arguments should be Matrix objects with three elements
    unit_1 = first_vector.normalized()
    unit_2 = second_vector.normalized()
    cross_vector = unit_1.cross(unit_2)
    return cross_vector.normalized()

In [97]:
t_0 = col((t[x_target],t[y_target],t[z_target]))
w_0 = col((wrist_x, wrist_y, wrist_z))

j3_s = (dh_parameter[a1] + dh_parameter[a2] * sin(joint_angles[q2])) # s_coordinate of joint_3 in base frame
j3_z = (dh_parameter[d1] + dh_parameter[a2] * cos(joint_angles[q2])) # s_coordinate of joint_3 in base frame

j3_x = j3_s * cos(joint_angles[q1])
j3_y = j3_s * sin(joint_angles[q1])

j3_0 = col((j3_x, j3_y, j3_z))

j3_0

⎡0.349839231908914⎤
⎢                 ⎥
⎢        0        ⎥
⎢                 ⎥
⎣1.99999998966145 ⎦

In [98]:
v = w_0 - j3_0
w = t_0 - w_0
# (v, w)
(v.subs(t), w.subs(t))

⎛⎡ 1.50002076809109  ⎤  ⎡0.453⎤⎞
⎜⎢                   ⎥  ⎢     ⎥⎟
⎜⎢        0.0        ⎥, ⎢  0  ⎥⎟
⎜⎢                   ⎥  ⎢     ⎥⎟
⎝⎣-0.0534199896614482⎦  ⎣  0  ⎦⎠

In [99]:
theta_5 = vector_cross_angle(v.subs(t), w.subs(t))
theta_5

0.0355977892046402

In [100]:
# c = v.cross(w)
c = v.cross(w).subs(t)
c

⎡         0         ⎤
⎢                   ⎥
⎢-0.0241992553166361⎥
⎢                   ⎥
⎣         0         ⎦

In [101]:
### Define functions for Rotation Matrices about x, y, and z given specific angle.

def rot_x(q):
    R_x = Matrix([[ 1,              0,        0],
                  [ 0,        cos(q),   -sin(q)],
                  [ 0,        sin(q),    cos(q)]])
    return R_x
    
def rot_y(q):
    R_y =  Matrix([[ cos(q),        0,  sin(q)],
                   [       0,        1,        0],
                   [-sin(q),        0,  cos(q)]])
    return R_y

def rot_z(q): 
    R_z = Matrix([[ cos(q), -sin(q),        0],
                  [ sin(q),  cos(q),        0],
                  [ 0,            0,        1]])
    return R_z


def composeRotations(rotation_list):
    rMat = eye(3)
    for rotation in rotation_list:
        rotation_axis, rotation_angle = rotation[0], rotation[1]
        rr = eval('rot_{}({})'.format(rotation_axis, rotation_angle))
        rMat = rr * rMat
    return rMat
    
    
def TF_matrix(rotation_list, translation=[0, 0, 0]):
    # rotation axis is element of {'x', 'y', 'z'}
#     transcol = Matrix([[translation[0]], [translation[1]], [translation[2]]])
    
#     rMat = eye(3)
#     for rotation in rot_sequence:
#         ax, degs = rotation[0], rotation[1]
#     rr = eval('rot_{}({})'.format(rotation_axis, rotation_angle))
#     rMat = rr * rMat
    rr = composeRotations(rotation_list)    
    rT = rr.row_join(col((translation[0], translation[1], translation[2])))
    rT = rT.col_join(Matrix([[0, 0, 0, 1]]))
    
    return rT

def DH_TF(alpha, a, d, q):
    # alpha, a, d, q = twist_angle, link_length, joint_angle, link_offset
    subT_x = TF_matrix([('x', alpha)], translation=[a, 0, 0])
    subT_z = TF_matrix([('z', q)], translation=[0, 0, d])
    return subT_x * subT_z


In [102]:
y, p, r = symbols('y, p, r')

v_hat = (v.subs(t)).normalized()
w_hat = (w.subs(t)).normalized()
vh = col((v_hat[0], v_hat[1], v_hat[2]), h=1)
wh = col((w_hat[0], w_hat[1], w_hat[2]), h=1)
v_hat, w_hat

⎛⎡ 0.999366465607479 ⎤  ⎡1.0⎤⎞
⎜⎢                   ⎥  ⎢   ⎥⎟
⎜⎢         0         ⎥, ⎢ 0 ⎥⎟
⎜⎢                   ⎥  ⎢   ⎥⎟
⎝⎣-0.0355902714125008⎦  ⎣ 0 ⎦⎠

In [103]:
R_r = composeRotations([('x', r)])
R_p = composeRotations([('y', p)])
R_y = composeRotations([('x', y)])

R_r, R_p, R_y

⎛⎡1    0        0   ⎤  ⎡cos(p)   0  sin(p)⎤  ⎡1    0        0   ⎤⎞
⎜⎢                  ⎥  ⎢                  ⎥  ⎢                  ⎥⎟
⎜⎢0  cos(r)  -sin(r)⎥, ⎢   0     1    0   ⎥, ⎢0  cos(y)  -sin(y)⎥⎟
⎜⎢                  ⎥  ⎢                  ⎥  ⎢                  ⎥⎟
⎝⎣0  sin(r)  cos(r) ⎦  ⎣-sin(p)  0  cos(p)⎦  ⎣0  sin(y)  cos(y) ⎦⎠

In [104]:
orientation = {r: pi/2,
               p: pi/2,
               y: pi/2
              }

R_wrist = R_y * R_p * R_r
R_wrist

⎡    cos(p)                  sin(p)⋅sin(r)                          sin(p)⋅cos
⎢                                                                             
⎢sin(p)⋅sin(y)   -sin(r)⋅sin(y)⋅cos(p) + cos(r)⋅cos(y)  -sin(r)⋅cos(y) - sin(y
⎢                                                                             
⎣-sin(p)⋅cos(y)  sin(r)⋅cos(p)⋅cos(y) + sin(y)⋅cos(r)   -sin(r)⋅sin(y) + cos(p

(r)            ⎤
               ⎥
)⋅cos(p)⋅cos(r)⎥
               ⎥
)⋅cos(r)⋅cos(y)⎦

In [111]:
w_rot = R_wrist.subs(orientation)
w_rot

⎡0  1  0 ⎤
⎢        ⎥
⎢1  0  0 ⎥
⎢        ⎥
⎣0  0  -1⎦

In [117]:
(w_0 + w).subs(t), (w_0 + (w_rot * w)).subs(t), N(pi/2)

⎛⎡2.30286⎤  ⎡1.84986⎤                 ⎞
⎜⎢       ⎥  ⎢       ⎥                 ⎟
⎜⎢   0   ⎥, ⎢ 0.453 ⎥, 1.5707963267949⎟
⎜⎢       ⎥  ⎢       ⎥                 ⎟
⎝⎣1.94658⎦  ⎣1.94658⎦                 ⎠

In [106]:
# T0_1 = DH_TF(alpha0, a0, d1, q1)
# T1_2 = DH_TF(alpha1, a1, d2, q2)
# T2_3 = DH_TF(alpha2, a2, d3, q3)
# T3_4 = DH_TF(alpha3, a3, d4, q4)
# T4_5 = DH_TF(alpha4, a4, d5, q5)
# T5_6 = DH_TF(alpha5, a5, d6, q6)
# T6_E = DH_TF(alpha6, a6, d7, q7)

T0_1 = DH_TF(alpha0, a0, d1, q1).subs(dh_parameter)
T1_2 = DH_TF(alpha1, a1, d2, q2).subs(dh_parameter)
T2_3 = DH_TF(alpha2, a2, d3, q3).subs(dh_parameter)
T3_4 = DH_TF(alpha3, a3, d4, q4).subs(dh_parameter)
T4_5 = DH_TF(alpha4, a4, d5, q5).subs(dh_parameter)
T5_6 = DH_TF(alpha5, a5, d6, q6).subs(dh_parameter)
T6_E = DH_TF(alpha6, a6, d7, q7).subs(dh_parameter)

o0_0 = col((0, 0, 0), 1)

o0_1 = T0_1 * o0_0
o2_0 = T1_2 * o1_0
o3_0 = T2_3 * o2_0

o0_0, o1_0, o2_0, o3_0

NameError: name 'o1_0' is not defined

In [ ]:
do_23 = o3_0 - o2_0

do_23, (do_23[0], do_23[1])

In [ ]:
o3_0[0]**2

In [120]:
k = Matrix(np.ndarray((2, 5, [3])))

TypeError: 'list' object cannot be interpreted as an integer